# Documentation

## Objective(s)
1. Ingest event log file data file into a consistent tabular format without error
2. Extract key features/attributes embedded in free text message log
3. Dump process logs into an SQL Server table for storage
4. Due to the different table structure for EventList and AlarmList data (AlarmList has redundant fields removed), the preprocessing script cannot be used interchangeably when appending data to the SQL Server Table.


# Initialisation

### Adjust Display Theme of Jupyter Notebook
Optional Step

Key Hotkeys:
1. Run cell: ctrl + enter
2. Delete Cell: DD
3. Undo Delete Cell: Z
4. Cut Cell: X
5. Copy Cell: C
6. Insert Cell Above: A
7. Insert Cell Below: B
8. Convert Cell to Code: Y
9. Convert Cell to Markdown: M

In [1]:
# install jupyterthemes
#!pip install jupyterthemes

# upgrade to latest version
#!pip install --upgrade jupyterthemes

In [2]:
#import jupyterthemes

In [3]:
# Adjust to Dark Theme
#jt -t chesterish

## Load Libraries

In [4]:
#pip install --user --upgrade pandas

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install schedule

In [6]:
# Import libraries
#import modin.pandas as pd
import pandas as pd
import numpy as np
import os
import pathlib
import datetime as dt
import time
import os # for manipulating file directories
import pyodbc # for sql operations
import sqlalchemy # for sql operations with pandas
import urllib # for defining sql connection parameters
import shutil # for transferring files between folders
#import re

## Set Options

In [7]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

In [8]:
# Choose whether to initialise db from scratch or not
initDB = True

In [9]:
# Trigger SQL Function Test
testMode = True
# Use this to reset table data especially when running tests
resetTableData = True

## Set Up Core Directories

In [10]:
# Check current directory location
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [11]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [12]:
# Create Directory for Output Files Generated
if not os.path.exists('dataCleaned'):
    os.makedirs('dataCleaned')
    
# Inspect files in directory
fileList = os.listdir()
fileList

['associationRules',
 'dataCleaned',
 'desktop.ini',
 'dummyLog',
 'HISevent_data',
 'Original Sample from 27 Oct 2020 (simplified).7z',
 'Repair Logs',
 'Sample from 27 Oct 2020 (OG).zip',
 'sample_data_allServer _(Jan2021).7z',
 'sample_data_allServer_raw&processed__(Jan2021).7z',
 'sample_data_HISevent.7z',
 'sample_data_test',
 'taggedOutput',
 'testLog',
 'testOutput']

In [13]:
# Location of Alarm and Normal Event Files
alarmLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\AlarmLogs'
eventLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs'
testLocAL = '\\sample_data_test\\batch 002\\AlarmLogs'
testLocAE = '\\sample_data_test\\batch 002\\EventLogs'

# Set variables
# Alarm Events: 'AL'  or 'TestAL' 
# All Events: 'AE' or 'TestAE' 

prefix = 'TestAE' 
batch = 'B001-'
serverEnv = 'CMS-'

## Access Files to be Processed

In [14]:
# Define root file directory folder where the files are being stored
if prefix == "AL":
    os.chdir(cwd + alarmLoc)
    #os.chdir(cwd + eventLoc)
elif prefix == "AE":
    os.chdir(cwd + eventLoc)
elif prefix == "TestAL":
    os.chdir(cwd + testLocAL)
else:
    os.chdir(cwd + testLocAE)
    
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\EventLogs'

In [15]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    #fileList += filenames
    
# Inspect data
len(fileList)

3263

In [16]:
testFile = fileList[0]
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\EventLogs\\EventListTest_1638320398_251753'

# Process Log Files

In [17]:
# Core Function for Cleaning (dependent on above functions)
# Excludes file saving
def cleaningScript_vector(fileInput, prefixInput = "TestAE", sqlServerExport = False):
    
    ##################################################################
    # Import Dependant Libraries
    ##################################################################
    
    import sys, os
    if 'pd' not in sys.modules:
        import pandas as pd
    if 'np' not in sys.modules:
        import numpy as np
    if 'dt' not in sys.modules:
        import datetime as dt
    #if 'os' not in sys.modules:
    #    import os as os
    
    ##################################################################
    # Initialise Dependant Function 1
    ##################################################################
    
    # Define function to remove free text message to facillitate formating later
    def replaceTextBetween1_series(originalTextSeries, delimeterA='Message:', delimterB='Theme:', replacementText=''):
        leadingText = originalTextSeries.str.split(delimeterA).str.get(0)
        trailingText = originalTextSeries.str.split(delimterB).str.get(1)

        return leadingText + replacementText + delimterB + trailingText
    
    
    ##################################################################
    # Initialise Dependant Function 2
    ##################################################################  
    
    def parseMessageField(originalTextSeries, delimeterA='Message:', delimterB='Theme:'):
        # Update v2.0
        # Remove leading delimiter (Delimiter A) from target string
        clip1 = originalTextSeries.str.split(delimeterA).str.get(1)
        # Remove lagging delimiter (Delimiter B) from target string
        clip2 = clip1.str.split(delimterB).str.get(0).str.strip()
        # Clean and split target string into constituent components
        clip3 = clip2.str.replace(";", "").str.split("$")
        
        
        # Delete redundant fields
        del clip1, clip2
        
        # Extract ASSET_ID_RAW from 1st Component of Clip 3
        ASSET_ID_RAW = clip3.str.get(0).str.strip()
        
        # Extract ASSET_DESCRIPTION from 2nd Component of Clip 3, a may be further delimited by a colon sign
        try:
            ASSET_DESCRIPTION = clip3.str.get(1).str.split(": ").str.get(0).str.strip()
        except: 
            ASSET_DESCRIPTION = None
        
         # Extract EVENT_DESCRIPTION from 2nd Component of Clip 3, a may be further delimited by a colon sign
        try:    
            EVENT_DESCRIPTION = clip3.str.get(1).str.split(": ").str.get(1).str.strip()
        except:
            EVENT_DESCRIPTION = None
        
         # Extract EVENT_STATUS from 3rd Component of Clip 3
        try:
            EVENT_STATUS = clip3.str.get(2).str.strip()
        except:
            EVENT_STATUS = None
        
        # Extract OPERATOR_INITIALS from 4th Component of Clip 4
        try:
            OPERATOR_INITIALS = clip3.str.get(3).str.strip()
        except:
            OPERATOR_INITIALS = None

        return ASSET_ID_RAW, ASSET_DESCRIPTION, EVENT_DESCRIPTION, EVENT_STATUS, OPERATOR_INITIALS
    
    ##################################################################
    # Initialise Dependant Function 3
    ##################################################################
    
    def parseMetaData(f_meta1, f_meta2):

        # Pre-process f_meta1
        f_meta1 = f_meta1.split('\n')[0].split('Begin Notification: ')[1].split(' ')

        # Pre-process f_meta2
        f_meta2 = f_meta2.split('End Notification: ')[1]

        # Get DATETIME_SEND
        DATETIME_SENT = str(f_meta1[0]) + ' ' + str(f_meta1[1])

        # Get TimeCode
        TIME_CODE = str(f_meta1[2]) + '.' + str(f_meta1[3])
        TIME_CODE2 = str(f_meta1[2]) + str(f_meta1[3])

        # Get DATETIME_RECEIVED
        DATETIME_RECEIVED = f_meta2

        return DATETIME_SENT, DATETIME_RECEIVED, TIME_CODE
    
    
    ##################################################################
    # Initialise Function
    ##################################################################
    
    # Define All Column Names (Especially for cases with an empty dataframe)
    col_names = [
                'ENTRY_CODE_SUFFIX', #00
                'ENTRY_CODE', #01
                'ALARM_ID', #02http://localhost:8888/notebooks/Documents/SBST%20Train%20IAMS%20Project/scripts/dataIngestion%20v2.1_packagedVer.ipynb#
                'USER_ID', #03
                'EQUIPMENT_NAME', #04
                'VALUE', #05
                'VALUE_STATE', #06
                'ACKNOWLEDGEMENT_REQUIRED', #07
                'SEVERITY', #08
                'HIDDEN', #09
                'THEME', #10
                'EQUIPMENT_DATE',  #11
                'ACQUISITION_DATE',  #12
                'SCS_TIME', #13
                'FUNCTIONAL_CATEGORY', #14
                'GEOGRAPHICAL_CATEGORY', #15
                'ENVIRONMENT', #16
                'USER1', #17
                'ASSET_ID_RAW', #18
                'ASSET_DESCRIPTION', #19
                'EVENT_DESCRIPTION', #20
                'EVENT_STATUS', #21
                'OPERATOR_INITIALS', #22
                'ASSET_DESC_CAT', #23
                'EVENT_DESC_CAT', #24
                'TrainID', #25
                'CarID', #26
                'ServiceID', #27
                'AssetClass', #28
                'AssetSubClass', #29
                #'LOG_TYPE',
                'DATETIME_SENT', #30
                'DATETIME_RECEIVED', #31
                'TIME_CODE' #32
                ]
    
    # Define Attribute Names for Header (main body excluding metadata)
    headerList_core = [              
                      'ENTRY_CODE_RAW', #0
                      'ENTRY_CODE_SUFFIX', #1
                      'ENTRY_CODE', #2
                      'ALARM_ID', #3
                      'AUTO_ID', #4
                      'USER_ID', #5
                      'EQUIPMENT_NAME', #0
                      'VALUE', #7
                      'VALUE_STATE', #8
                      'ACKNOWLEDGEMENT_REQUIRED', #9
                      'SEVERITY', #10
                      'SHELVE', #11
                      'HIDDEN', #12
                      'THEME', #13
                      #'MESSAGE', #XX
                      'EQUIPMENT_DATE', #14
                      'EQUIPMENT_DATE0', #15
                      'ACQUISITION_DATE', #16
                      'ACQUISITION_DATE0', #17
                      'SCS_TIME', #18
                      'SCS_TIME0', #19
                      'FUNCTIONAL_CATEGORY', #20
                      'GEOGRAPHICAL_CATEGORY', #21
                      'ACKNOWLEDGEMENT_AUTOPOINTER', #22
                      'ENVIRONMENT', #23
                      'USER1', #24
                      'USER2', #25
                      'DSS_EVENT_TYPE' #26
                 ]
     
    ##################################################################
    # Ingest File
    ##################################################################
    
    # Define File Name
    fileName = fileInput
    
    # Read in file
    file = open(fileName, 'r')
        
    # Convert file contents to a list
    fileContents = list(file)

    # Close file
    file.close()

               
    ##################################################################
    # Extract Key Components in Ingested File
    ##################################################################
    
    # Extract row containing meta data of file sent datetime
    f_meta1 = fileContents[1]
    
    # Extract row containing meta data of file received datetime
    f_meta2 = fileContents[-1]
        
    # Remove redundant list elements
    del fileContents[0:3], fileContents[-1]
    
    # Error catch if this is the first log instance (empty log)
    if (f_meta1.find("first notification on online") > -1):
        # Create an Empty Dataframe with all the headers to match other dataframes to be appended
        df = pd.DataFrame(columns = col_names)
        
        # Delete redundant variables
        del f_meta1, f_meta2
    else:  
        # Load File
        df = pd.DataFrame(fileContents,columns=['rawData'])
        #print(df["rawData"][0])
        
        #print(ASSET_ID_RAW)
        #ASSET_ID_RAW = ASSET_ID_RAW.str.encode('ascii', 'ignore').str.decode() # TEST
        
        
        ##################################################################
        # Process Main Body (Exclude Free Text Message Field)
        ##################################################################
        
        # Exception handling for cases where the log entries are corrupted and split into 2 rows per entry
        # instead of the usual single row;
        # these cases tend to have a substring of unicode special characters marking the breakpoints

        # corruption check
        df["rawData"] = df["rawData"].str.strip() # remove non-printable char in string
        check_val_head = sum(df["rawData"].str.endswith("(.UàÑ"))
        check_val_tail = sum(df["rawData"].str.startswith("º÷"))
        
        print(check_val_head)
        print(check_val_tail)
        
        if (check_val_head == check_val_tail) & (check_val_head > 0) & (check_val_tail > 0):
            # If event log entry is corrupted, do this...
            
            # Separate breakpoint 1
            df_head = df.loc[df["rawData"].str.endswith("(.UàÑ")].copy()
            
            #print(df_head["rawData"][0])
            df_head["rawData"] = df_head["rawData"].str.split(pat = ".UàÑ", n = 1, expand = True)[0]
            df_head["rawData"] = df_head["rawData"].str.replace(pat = ";(", repl = "", regex = False)
            #df_head["rawData"] = df_head["rawData"].str.replace("(.UàÑ", regex = False)
            
            # Separate breakpoint 2
            df_tail = df.loc[df["rawData"].str.startswith("º÷")].copy()
            # drop first 3 char which has the unicode substring, direct targeting is not possible as that throws an error
            df_tail["rawData"] = df_tail["rawData"].str[3:] 
            
            # Reset index of dataframes
            df_head = df_head.copy().reset_index()
            df_tail = df_tail.copy().reset_index()
            
            # Join and repair corrupted entries which has been broken up into 2 rows
            df_head = df_head["rawData"] + df_tail["rawData"]

            # Filter out corrupted entires
            df = df.loc[(df["rawData"].str.endswith("(.UàÑ") == False)].copy()
            df = df.loc[(df["rawData"].str.startswith("º÷") == False)].copy()
            df = df.squeeze()
                        
            # Add back repaired corrupted entries
            df = df.append(df_head, ignore_index=True)
            df = pd.DataFrame(df, columns = ['rawData'])
            
            # Delete redundant variables
            del df_head, df_tail
            
            # Remove free text string "message" from log string
            df['text0'] = replaceTextBetween1_series(df["rawData"])
            
        else:
            # If event log entry is ok, do this...
            
            # Remove free text string "message" from log string
            df['text0'] = replaceTextBetween1_series(df["rawData"])
        
        # Delete redundant variables
        del check_val_head, check_val_tail
        
        # Delimit Columns
        df[headerList_core] = df['text0'].str.split(expand = True)

        # Merge Date and Time columns
        df['EQUIPMENT_DATE'] = df['EQUIPMENT_DATE'] + "." + df['EQUIPMENT_DATE0'].astype(str)
        df['ACQUISITION_DATE'] = df['ACQUISITION_DATE'] + "." + df['ACQUISITION_DATE0'].astype(str)
        df['SCS_TIME'] = df['SCS_TIME'] + "." + df['SCS_TIME0'].astype(str)
        
        # Extract Entry Code Suffix
        df['ENTRY_CODE_SUFFIX'] = df["ENTRY_CODE_RAW"].str.extract(r"\(""([+|=|-])")

        # Drop redundant columns
        df = df.drop(columns=['ENTRY_CODE_RAW', 
                              'EQUIPMENT_DATE0', 
                              'ACQUISITION_DATE0', 
                              'SCS_TIME0', 
                              'text0'])

        # Clean up values of each row by removing variable name prefixed to each value
        n = 3
        nEnd = df.shape[1]

        #df.iloc[::,n] = df.iloc[::,n].str.split(':', expand = True)[1]
        # Update v2.1
        while (n < nEnd):
            df.iloc[::,n] = df.iloc[::,n].str.split(':', n = 1, expand = True)[1]
            n = n + 1

        # Delete redundant variables
        del n, nEnd

        # Drop redundant entries caused by entry deletion due to server memory limitations
        # Marked by "-" suffix in ["ENTRY_CODE_SUFFIX"]
        df = df.loc[df['ENTRY_CODE_SUFFIX'] != "-"].reset_index(drop=True)
        
        
        ##################################################################
        # Check if the remaining dataframe is empty or not
        ##################################################################

        if df.shape[0] == 0:
            #print("Empty Dataframe " + fileName)
            # Create an Empty Dataframe with all the headers to match other dataframes to be appended
            df  = pd.DataFrame(columns = col_names) 
        else:
            #print("Continue file processing")


            ##################################################################
            # Process Free Text Message Field
            ##################################################################

            # Remove free text string "message" from log string
            #del df['MESSAGE']
                        
            # Parse Free Text Message Log
            
            df["ASSET_ID_RAW"], df["ASSET_DESCRIPTION"], df["EVENT_DESCRIPTION"], df["EVENT_STATUS"], df["OPERATOR_INITIALS"] = parseMessageField(df["rawData"])
            
            ###############################################################
    
            # Reorder values of exception cases of EVENT_DESCRIPTION and ASSET_DESCRIPTION
            # EVENT_DESCRIPTION is prioritised
            try:
                df.loc[((df["EVENT_DESCRIPTION"] == None) | 
                       (df["EVENT_DESCRIPTION"] == "") | 
                       df["EVENT_DESCRIPTION"].isna()), 
                       "EVENT_DESCRIPTION"] = df["ASSET_DESCRIPTION"]

                df.loc[(df["EVENT_DESCRIPTION"] == df["ASSET_DESCRIPTION"]), 
                       "ASSET_DESCRIPTION"] = None                
            except:
                pass

            ###############################################################
            # Clean up for "GWS Broadcast" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("msg", na = False, regex = False)) & 
                       (df["EVENT_DESCRIPTION"].str.contains("Gws", na = False, regex = False)), "ASSET_DESCRIPTION"] = "GWS Broadcast"
            except:
                pass  

            ###############################################################
            # Clean up for "NelVisu" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("NelVisu", na = False, regex = False)), "ASSET_DESCRIPTION"] = "NelVisu"
            except:
                pass

            ###############################################################
            # Clean up for "Train Radio" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("TR___", na = False, regex = False)) &
                       (df["EVENT_DESCRIPTION"].str.contains("radio", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Train Radio"
            except:
                pass

            ###############################################################
            # Clean up for "Trainborne Camera" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Trainborne Camera", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Trainborne Camera"
            except:
                pass

            ###############################################################
            # Clean up for "Trainborne Quad" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Trainborne Quad", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Trainborne Quad"
            except:
                pass

            ###############################################################
            # Clean up for "Tunnel Light" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Tunnel Light", na = False, regex = False)) & 
                        ((df["ASSET_DESCRIPTION"].isna()) | 
                         (df["ASSET_DESCRIPTION"] == None) | 
                         (df["ASSET_DESCRIPTION"] == "")), "ASSET_DESCRIPTION"] = "Tunnel LTG"
            except:
                pass   

            ###############################################################
            # Clean up for "Control Take Over" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Control Take Over for", na = False, regex = False)), "ASSET_DESCRIPTION"] = df["EVENT_DESCRIPTION"].str.extract(r"Control Take Over for (\w+) ")
                
            except:
                pass
            
            ###############################################################
            # Clean up for "Close Control" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Close Control", na = False, regex = False)) & 
                      ((df["ASSET_DESCRIPTION"].isna()) | 
                       (df["ASSET_DESCRIPTION"] == None) | 
                       (df["ASSET_DESCRIPTION"] == "")
                      ), "ASSET_DESCRIPTION"] = "Traction Control"
            except:
                pass    

            ###############################################################
            # Get Asset Description Category (Remove Stopwords)
            # Define Key Location Names
                        
            locNamesList = {
                        'NED', #001
                        'FRP', #002
                        'SKG', #003
                        'HGN', #004
                        'KVN', #005
                        'SER', #006
                        'HBF', #007
                        'DBG', #008
                        'OTP', #009
                        'CNT', #010
                        'LTI', #011
                        'CQY', #012
                        'BGK', #013
                        'OCC', #014
                        'WLH', #015
                        'PTP', #016
                        'BNK', #017
                        'PGL', #018
                        'TUNNEL', #019
                        'Sector', #020
                        'Concourse', #021
                        'Mezzaninne', #022
                        'Mid-Landing Entrance', #023
                        'AL', #024
                        'Dirty Area', #025
                        'IAP', #026
                        '1st Storey', #027
                        '2nd Storey', #028
                        '3rd Storey', #029
                        'B1', #030
                        'B2', #031
                        'B3', #032
                        'Entrance', #033
                        'Mid Landing', #034
                        'Mid-Landing', #035
                        'Subway', #036
                        'Underpass Link', #037
                        "Underpass To EXT'G  STN", #038
                        "1st", #039 
                        "2nd", #040 
                        "SUBLOCATIONN", #041 
                        "SUBLOCATIONS", #042 
                        "North End", #043 
                        "South End", #044 
                        "South Adjacent", #045 
                        "North Adjacent", #046 
                        "Mezzanine", #047 
                        "Linkway", #048 
                        "Smoke Free Lobby", #049 
                        "Storey", #050 
                        "Underpass to EXT'G STN", #051 
                        "-SUBLOCATION", #052 
                        "SUBLOCATION-" #053 
                        }
            
            
            locNamesVal = [
                        '', #001
                        '', #002
                        '', #003
                        '', #004
                        '', #005
                        '', #006
                        '', #007
                        '', #008
                        '', #009
                        '', #010
                        '', #011
                        '', #012
                        '', #013
                        '', #014
                        '', #015
                        '', #016
                        '', #017
                        '', #018
                        '', #019
                        '', #020
                        'SUBLOCATION', #021
                        'SUBLOCATION', #022
                        'SUBLOCATION', #023
                        'SUBLOCATION', #024
                        'SUBLOCATION', #025
                        'SUBLOCATION', #026
                        'SUBLOCATION', #027
                        'SUBLOCATION', #028
                        'SUBLOCATION', #029
                        '', #030
                        '', #031
                        '', #032
                        'SUBLOCATION', #033
                        '', #034
                        'SUBLOCATION', #035
                        'SUBLOCATION', #036
                        'SUBLOCATION', #037
                        'SUBLOCATION', #038
                        "", #039 
                        "", #040 
                        "SUBLOCATION", #041 
                        "SUBLOCATION", #042 
                        "", #043 
                        "", #044 
                        "", #045 
                        "", #046 
                        "SUBLOCATION", #047 
                        "SUBLOCATION", #048 
                        "SUBLOCATION", #049 
                        "SUBLOCATION", #050 
                        "SUBLOCATION", #051 
                        "", #052 
                        "" #053 
                        ]
            
            
            # Remove Location Names    

            df["ASSET_DESC_CAT"] = df["ASSET_DESCRIPTION"]    
            try: # Error catch if the entire column is empty
                #df= df.replace({"ASSET_DESC_CAT": locNames}, regex = True) # not compatible with modin; slower than list method
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].replace(regex = locNamesList, value = locNamesVal)
            except:
                pass
            
            ###############################################################
            # Get Asset Description Category (Remove Numbers)
            # Remove Numbers
            df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace(r'\d+', '', regex = True)

            # Account for exceptions (Update v2.0)
            # try and except clauses to catch null cases and cases where there are completely null cases
            try:
                df['ASSET_DESC_CAT'].str.replace(" kV", "22 kV", regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'].str.replace("at KV SW", "at 22 kV SW", regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'].str.replace("DC  V", "DC 1500 V", regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace('SUBLOCATION SUBLOCATION', 'SUBLOCATION', regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace('( ', '(', regex = False)
            except:
                pass
            try:
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].str.replace(r'\A(: )','', regex = True)
            except:
                pass    
            try:
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].str.replace('Cameras','Camera', case = False, regex = False)
            except:
                pass
            
            try:
                df.loc[df['ASSET_DESCRIPTION'] == "22 kV Feeder CB", "ASSET_DESC_CAT"] = "22 kV Feeder CB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "22 kV Loop CB", "ASSET_DESC_CAT"] = "22 kV Loop CB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "22 kV Rectifier CB", "ASSET_DESC_CAT"] = "22 kV Rectifier CB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "DC 1500 V Backup HSCB", "ASSET_DESC_CAT"] = "DC 1500 V Backup HSCB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "DC 1500 V Bus Section", "ASSET_DESC_CAT"] = "DC 1500 V Bus Section"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "DC 1500 V Feeder CB", "ASSET_DESC_CAT"] = "DC 1500 V Feeder CB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "DC 1500 V Rectifier CB", "ASSET_DESC_CAT"] = "DC 1500 V Rectifier CB"
            except:
                pass

            try:
                df.loc[df['ASSET_DESCRIPTION'] == "DC 1500 V Inverter CB", "ASSET_DESC_CAT"] = "DC 1500 V Inverter CB"
            except:
                pass

            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("CCTV Controller Power Supply", na = False, regex = False), "ASSET_DESC_CAT"] = "CCTV Controller Power Supply"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("CBN Access Multiplexer", na = False, regex = False), "ASSET_DESC_CAT"] = "CBN Access Multiplexer"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("CI Gas Panel", case = False, na = False, regex = False), "ASSET_DESC_CAT"] = "CI Gas Panel"
            except:
                pass
            try: # (Update 2.1)
                df.loc[df["ASSET_DESC_CAT"].str.contains("RI Gas Panel", case = False, na = False, regex = False), "ASSET_DESC_CAT"] = "RI Gas Panel"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("CROSS-CONNECT ACCESS Multiplexer", na = False, regex = False), "ASSET_DESC_CAT"] = "CROSS-CONNECT ACCESS Multiplexer"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Electrically Supervised Valve", na = False, regex = False), "ASSET_DESC_CAT"] = "Electrically Supervised Valve"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Hosereel Pump", na = False, regex = False), "ASSET_DESC_CAT"] = "Hosereel Pump"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Level Fire Shutter", na = False, regex = False), "ASSET_DESC_CAT"] = "Level Fire Shutter"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Level Roller Shutter", na = False, regex = False), "ASSET_DESC_CAT"] = "Level Roller Shutter"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Main Fire Alarm Panel", na = False, regex = False), "ASSET_DESC_CAT"] = "Main Fire Alarm Panel"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Traffic Direction", case = False, na = False, regex = False), "ASSET_DESC_CAT"] = "Traffic Direction"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Tunnel LTG Ctrl Panel", case = False, na = False, regex = False), "ASSET_DESC_CAT"] = "Tunnel LTG Ctrl Panel"
            except:
                pass
            try:
                df.loc[df["ASSET_DESC_CAT"].str.contains("Zone -", case = False, na = False, regex = False), "ASSET_DESC_CAT"] = "ZONE SUBLOCATION"
            except:
                pass
            
            # Remove additional locations (Update v2.0)
            # try and except clauses to catch null cases and cases where there are completely null cases
            try:
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].str.split(" at ", n = 1, expand = True)[0]
            except:
                pass
            try:
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].str.split(" for ", n = 1, expand = True)[0]
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace('SUBLOCATION-SUBLOCATION', 'SUBLOCATION', regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace('SUBLOCATION-', 'SUBLOCATION', regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace('-SUBLOCATION', 'SUBLOCATION', regex = False)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace(r'( at)$', '', case = False, regex = True)
            except:
                pass
            try:
                df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace(r'( for)$', '', case = False, regex = True)
            except:
                pass
            try:
                    df['ASSET_DESC_CAT'] = df['ASSET_DESC_CAT'].str.replace(r'^(:)', '', regex = True)
            except:
                pass
            
            # Remove redundant white spaces and leading/trailing spaces  (Update v2.0)
            # try and except clauses to catch null cases and cases where there are completely null cases
            try:
                df["ASSET_DESC_CAT"] = df["ASSET_DESC_CAT"].str.strip().str.replace(r'\s+', ' ', regex = True)  
            except:
                pass

            ###############################################################
            
            # Create "EVENT_DESCRIPTION" field
            df["EVENT_DESC_CAT"] = df["EVENT_DESCRIPTION"]

            # Remove Location Names (NEW UPDATE)     
            try: # Error catch if the entire column is empty
                #df = df.replace({"EVENT_DESC_CAT": locNames}, regex = True) # Does not work with Modin; slower than list method
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].replace(regex = locNamesList, value = "")
            except:
                pass
            
            ###############################################################
            # Perform preliminary cleaning (Update v2.0)
            # try and except clauses to catch null cases and cases where there are completely null cases
            
            try: 
                # Get Event Description Category (Remove Numbers)
                # Remove Numbers
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'\d+', '', regex = True)
            except:
                pass
            
            try:
                # Get Event Description Category (Remove Redundant White Spaces)
                # Remove redundant white spaces    
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.strip().str.replace(r'\s+', ' ', regex = True)
            except:
                pass
            
            ###############################################################
            # Get Event Description Category (Account for Misc Exceptions) (Update v2.0)
            # try and except clauses to catch null cases and cases where there are completely null cases
            # Account for Exceptions
            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("logged", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains("Operator", na = False, regex = False)) &
                       (df['EVENT_DESC_CAT'].str.contains("NelVisu", na = False, regex = False)),
                       "EVENT_DESC_CAT"
                      ] = "Operator Logged In/Out of NelVisu"
            except:
                pass
            
            try:
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' /, /...', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' /', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'_:', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'([.]+){2}', '', regex = True)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'(_+){2}', '', regex = True)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'::', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' : ', ': ', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'@n', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' ,', ',', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'< >', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'-:', ':', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'^(:)', '', regex = True)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'( -)\S', ' - ', regex = True)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' )', ')', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'()', '', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' ump Rm', ' Pump Rm', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' latform', ' Platform', regex = False)
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r'( )+', ' ', regex = True) 
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace('SUBLOCATION-SUBLOCATION', 'SUBLOCATION', regex = False) 
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace('SUBLOCATION-', 'SUBLOCATION', regex = False) 
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace('-SUBLOCATION', 'SUBLOCATION', regex = False) 
            except:
                pass
            
            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains(" at ", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains(": ", na = False, regex = False)), "EVENT_DESC_CAT"] = df['EVENT_DESC_CAT'].str.split("at", 1, expand = True)[0] + ": " + df['EVENT_DESC_CAT'].str.split(": ", 1, expand = True)[1]
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.replace(r' : ', ': ', regex = False)
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Gws", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains("msg in", na = False, regex = False)), "EVENT_DESC_CAT"] = df['EVENT_DESC_CAT'].str.split("msg in", 1, expand = True)[0] + "msg in SUBLOCATION"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Gws", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains("bcast in", na = False, regex = False)), "EVENT_DESC_CAT"] = df['EVENT_DESC_CAT'].str.split("bcast in", 1, expand = True)[0] + "bcast in SUBLOCATION"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Train", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains("Car", na = False, regex = False)) &
                       (df['EVENT_DESC_CAT'].str.contains("assigned", na = False, regex = False)) &
                       (df['EVENT_DESC_CAT'].str.contains("Manoeuvre", na = False, regex = False)), "EVENT_DESC_CAT"] = "Manoeuvre assigned to Train Car"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Train", na = False, regex = False)) & 
                       (df['EVENT_DESC_CAT'].str.contains("Car", na = False, regex = False)) &
                       (df['EVENT_DESC_CAT'].str.contains("abandoned", na = False, regex = False)) &
                       (df['EVENT_DESC_CAT'].str.contains("Manoeuvre", na = False, regex = False)), "EVENT_DESC_CAT"] = "Manoeuvre abandoned by Train Car"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESCRIPTION'].str.contains("Display of Free-Text", na = False, regex = False)) | 
                       (df['ASSET_DESCRIPTION'].str.contains("Display of Free-Text", na = False, regex = False))
                       , "EVENT_DESC_CAT"] = "Display of Free-Text"
            except:
                pass
            
            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("DVA version mismatch", na = False, regex = False)), "EVENT_DESC_CAT"] = "DVA version mismatch"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Automatic hand-over", na = False, regex = False)), "EVENT_DESC_CAT"] = "Automatic hand-over"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Automatic Hold Applied", na = False, regex = False)), "EVENT_DESC_CAT"] = "Automatic Hold Applied"
            except:
                pass

            try:
                df.loc[(df['EVENT_DESC_CAT'].str.contains("Communication between", na = False, regex = False)), "EVENT_DESC_CAT"] = "Communication between Nodes"
            except:
                pass
            
            # Track and Train Direction (NEW UPDATE)
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace("S/B ", "", regex = False)
            except:
                pass
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace("N/B ", "", regex = False)
            except:
                pass
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace(" S ", "", regex = False)
            except:
                pass
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace(" N ", "", regex = False)
            except:
                pass
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace(" N-N ", "", regex = False)
            except:
                pass
            try:
                df["EVENT_DESC_CAT"] = df["EVENT_DESC_CAT"].str.replace(" S-S ", "", regex = False)
            except:
                pass
            
            try:
                df['EVENT_DESC_CAT'] = df['EVENT_DESC_CAT'].str.strip().str.replace(r'\s+', ' ', regex = True)
            except:
                pass
            
            # Automatic Hold Applied (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Automatic Hold applied to TrainCar stalled", regex = False)), "EVENT_DESC_CAT"] = "Automatic Hold applied to TrainCar stalled"
            except:
                pass
            # Control Hand Over (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for ECS - Environmental Control System", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for ECS - Environmental Control System"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for ECS - Smoke Extraction System", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for ECS - Smoke Extraction System"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for ECS - Tunnel Ventilation System", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for ECS - Tunnel Ventilation System"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for SIG - Control Train ATC", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for SIG - Control Train ATC"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for SIG - Platform Equipment", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for SIG - Platform Equipment"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for SIG - Track Side Equipment", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for SIG - Track Side Equipment"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for TrainBorne CCTV", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for TrainBorne CCTV"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for TrainBorne PA", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for TrainBorne PA"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for TrainBorne PEC", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for TrainBorne PEC"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Hand Over for TrainBorne PIS/VPIS", regex = False)), "EVENT_DESC_CAT"] = "Control Hand Over for TrainBorne PIS/VPIS"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Take Over for All Functions", regex = False)), "EVENT_DESC_CAT"] = "Control Take Over for All Functions"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Control Take Over for PIS - Passenger Information", regex = False)), "EVENT_DESC_CAT"] = "Control Take Over for PIS - Passenger Information"
            except:
                pass
            # Operator Calls (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("accepts a PEC call", regex = False)), "EVENT_DESC_CAT"] = "OPERATOR accepts a PEC call"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("terminates all PEC calls", regex = False)), "EVENT_DESC_CAT"] = "OPERATOR terminates all PEC calls"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("terminates PEC call", regex = False)), "EVENT_DESC_CAT"] = "OPERATOR terminates PEC call"
            except:
                pass
            # Free all paths to Stations (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Free all paths for Station", regex = False)), "EVENT_DESC_CAT"] = "Free all paths for Station"
            except:
                pass
            # Gama status request (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Gama Status Request For an Atc", regex = False)), "EVENT_DESC_CAT"] = "Gama Status Request For an Atc"
            except:
                pass
            # NelVisu Password Change (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("change password on NelVisu", regex = False)), "EVENT_DESC_CAT"] = "OPERATOR change password on NelVisu"
            except:
                pass
            # Track-Side Atc Status Request (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Track-Side Atc Status Request for An Atc", regex = False)), "EVENT_DESC_CAT"] = "Track-Side Atc Status Request for An Atc"
            except:
                pass
            # Train Location (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Train found at", regex = False)) &
                       (df["EVENT_DESC_CAT"].str.contains("instead of Train", regex = False)), "EVENT_DESC_CAT"] = "Train found at SUBLOCATION instead of Train"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Train still not a Man RTL", regex = False)) &
                       (df["EVENT_DESC_CAT"].str.contains("origin after wait period", regex = False)), "EVENT_DESC_CAT"] = "Train still not a Man RTL origin after wait period"
            except:
                pass
            # Timetable (NEW UPDATE)
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Timetable", regex = False)) &
                       (df["EVENT_DESC_CAT"].str.contains("download", regex = False)), "EVENT_DESC_CAT"] = "Timetable download"
            except:
                pass
            try:
                df.loc[(df["EVENT_DESC_CAT"].str.contains("Timetable", regex = False)) &
                       (df["EVENT_DESC_CAT"].str.contains("successfully autoloaded", regex = False)), "EVENT_DESC_CAT"] = "Timetable successfully autoloaded"
            except:
                pass
            
            
            ###############################################################
            # Extract Train Information (Update v2.0)
            # Error in the preprocessing of "EVENT_DESCRIPTION" & "ASSET_ID_RAW" cause some values to be missed out (WIP)
            # First case TrainID
            df["TrainID1"] = df["EVENT_DESCRIPTION"].str.extract(r"Train (\d+)") 
            # 2nd case TrainID
            df["TrainID2"] = df["ASSET_ID_RAW"].str.extract(r"TR___(\d+)") 
            # 3rd case TrainID
            df["TrainID3"] = df["EVENT_DESCRIPTION"].str.extract(r"TR___(\d+)") 
            # Combine TrainID results
            df["TrainID"] = (df["TrainID1"].astype(str) +
                             df["TrainID2"].astype(str) +
                             df["TrainID3"].astype(str)).str.replace("nan", "",regex=None)
            df["TrainID"] = df["TrainID"].replace("", np.nan)
            del df["TrainID1"], df["TrainID2"], df["TrainID3"]
            
            # First case CarID
            df["CarID1"] = df["EVENT_DESCRIPTION"].str.extract(r"Car (\d+)") 
            # 2nd case CarID & ServiceID
            df[["CarID2", "ServiceID"]] = df["EVENT_DESCRIPTION"].str.extract(r"cars (\d+)/(\d+)") 
            # Combine CarID results
            df["CarID"] = (df["CarID1"].astype(str) +
                             df["CarID2"].astype(str)).str.replace("nan", "",regex=None)
            df["CarID"] = df["CarID"].replace("", np.nan)
            del df["CarID1"], df["CarID2"]
            

            ###############################################################
            # Extract Asset Information
              
            df["AssetClass"] = df["ASSET_ID_RAW"] 
            
            # Remove Location Names  
            try: # Error catch if entire column is empty
                #df = df.replace({"AssetClass": locNames}, regex = True) # Does not work with Modin; slower than list method
                df["AssetClass"] = df["AssetClass"].replace(regex = locNamesList, value = "")
            except:
                pass
            
            # Remove Numbers
            df['AssetClass'] = df['AssetClass'].str.replace(r'\d+', '', regex = True)

            # Remove Exceptions
            try:
                df.loc[df['AssetClass'].str.contains("TRACTION", na = False, regex = False), 'AssetClass'] = "TRACTION/TRACTION"
            except:
                pass

            try:
                df.loc[(df['AssetClass'].str.contains("TUNNEL", na = False, regex = False)) & 
                   (df['AssetClass'].str.contains("LIGHT", na = False, regex = False)), 'AssetClass'] = "TUNNEL/LIGHT"
            except:
                pass

            # Clean up string prior to delimiting
            df['AssetClass'] = df['AssetClass'].str.replace(r'\A(_)', '', regex = True)
            df['AssetClass'] = df['AssetClass'].str.replace(r'(_)\Z', '', regex = True)
            df['AssetClass'] = df['AssetClass'].str.replace(r'_+', '/', regex = True)
            
            # Get AssetSubClass
            #df['AssetSubClass'] = df['AssetClass'].str.split("/").str[-1] # this method doesn't work with modin
            df['AssetSubClass'] = df['AssetClass'].str.extract(r'/(\w+)$')

            # Get AssetClass
            #df['AssetClass'] = df['AssetClass'].str.split("/").str[0] # this method doesn't work with modin
            df['AssetClass'] = df['AssetClass'].str.extract(r'(\w+)/')[0]
            
            # Delete redundant variables
            del df["rawData"]

            ##################################################################
            # Process Meta Data
            ##################################################################

            try:
                metaData = parseMetaData(f_meta1, f_meta2)
            except:
                metaData = ['01/01/00 00:00:00.000','01/01/00 00:00:00.000',0]


            # Delete redundant variables
            del f_meta1, f_meta2, fileContents

            # Insert Meta Data
            #df['LOG_TYPE'] = prefixInput
            df['DATETIME_SENT'] = metaData[0]
            df['DATETIME_RECEIVED'] = metaData[1]
            df['TIME_CODE'] = metaData[2]

            # Delete redundant variables
            del metaData


            ##################################################################
            # Format Data Type & Perform Final Clean Up
            ################################################################## 
            
            # Clean up operator initials (Update v2.0)
            try:
                df.loc[df["OPERATOR_INITIALS"].str.contains("(", na=False, regex=False), "OPERATOR_INITIALS"] = np.nan
            except:
                pass
            
            
            # Standardise null values
            df = df.replace("", np.nan).fillna(value=np.nan)          
            
            # Preliminary Formatting
            df = df.convert_dtypes()
            
            # Convert binary values to Boolean True/False
            try:
                df.loc[df["ACKNOWLEDGEMENT_REQUIRED"] == 0,"ACKNOWLEDGEMENT_REQUIRED"] = False
            except:
                pass
            try:
                df.loc[df["ACKNOWLEDGEMENT_REQUIRED"] == 1, "ACKNOWLEDGEMENT_REQUIRED"] = True
            except:
                pass
            
            try:
                df.loc[df["HIDDEN"] == 0, "HIDDEN"] = False
            except:
                pass            
            try:
                df.loc[df["HIDDEN"] == 1, "HIDDEN"] = True
            except:
                pass
            
            # Check data types
            df.dtypes# Preliminary Formatting
            df = df.convert_dtypes()

            # Format fields
            #df[''] = df[''].astype('')
            convert_dict = {'ALARM_ID': 'str',
                            'AUTO_ID': 'str',
                            'USER_ID': 'str',
                            'VALUE': 'str',
                            'VALUE_STATE': 'category',
                            'ACKNOWLEDGEMENT_REQUIRED': 'bool',
                            'SEVERITY': 'category',
                            'SHELVE': 'category',
                            'HIDDEN': 'bool',
                            'THEME': 'category',
                            'FUNCTIONAL_CATEGORY': 'category',
                            'GEOGRAPHICAL_CATEGORY': 'category',
                            'EQUIPMENT_DATE': 'float',
                            'ACQUISITION_DATE': 'float',
                            'SCS_TIME': 'float',
                            'ASSET_ID_RAW': 'str',
                            'ASSET_DESCRIPTION': 'str',
                            'EVENT_DESCRIPTION': 'str',
                            'EVENT_STATUS': 'category',
                            'OPERATOR_INITIALS': 'str',
                            'ASSET_DESC_CAT': 'category',
                            'EVENT_DESC_CAT': 'category',
                            'TrainID': 'str',
                            'CarID': 'str',
                            'ServiceID': 'str',
                            'AssetClass': 'category',
                            'AssetSubClass': 'category',
                            'TIME_CODE': 'float'
                           }
            df = df.astype(convert_dict)
            
            # Delete redundant variables
            del convert_dict           

            # Format String Values to Datetime Format
            df['DATETIME_SENT'] = pd.to_datetime(df['DATETIME_SENT'], format='%m/%d/%y %H:%M:%S.%f', errors='coerce')
            df['DATETIME_RECEIVED'] = pd.to_datetime(df['DATETIME_RECEIVED'], format='%m/%d/%y %H:%M:%S.%f', errors='coerce')

            # This code block could be redundant as the fields are not being used
            # Format Unix Time Values to Datetime Format in Local Time (GMT+8)
            # 19 digits unix time (without decimal place): nanosecond resolution (ns)
            # 16 digits unix time (without decimal place): microsecond resolution (us)
            # 13 digits unix time (without decimal place): millisecond resolution (ms)
            # 10 digits unix time (without decimal place): second resolution (s)
            # The values seem to be 9 hours behind (standard unix time should be pegged to GMT)
            timeAdjust = 8
            df['EQUIPMENT_DATE'] = pd.to_datetime(df['EQUIPMENT_DATE'], unit='s') + pd.Timedelta(hours = timeAdjust)
            df['ACQUISITION_DATE'] = pd.to_datetime(df['ACQUISITION_DATE'], unit='s') + pd.Timedelta(hours = timeAdjust)
            df['SCS_TIME'] = pd.to_datetime(df['SCS_TIME'], unit='s') + pd.Timedelta(hours = timeAdjust)
            df['TIME_CODE'] = pd.to_datetime(df['TIME_CODE'], unit='s') + pd.Timedelta(hours = timeAdjust) 

            # Drop redundant columns
            df = df.drop(columns=[
                                    'AUTO_ID', 
                                    'SHELVE', 
                                    #'EQUIPMENT_DATE', 
                                    #'ACQUISITION_DATE', 
                                    #'SCS_TIME', 
                                    'ACKNOWLEDGEMENT_AUTOPOINTER', 
                                    'USER2', 
                                    'DSS_EVENT_TYPE'
                                   ])
    
    # Preliminary tagging of alarm events
    if (prefixInput == "AL") | (prefixInput == "TestAL"):
        df["isAlarm"] = True
    else:
        df["isAlarm"] = False
        
    # Convert boolean values to 1 and 0 for SQL Server Export
    if sqlServerExport == True:
        df["ACKNOWLEDGEMENT_REQUIRED"] = df["ACKNOWLEDGEMENT_REQUIRED"].astype(int)
        df["HIDDEN"] = df["HIDDEN"].astype(int)
        df["isAlarm"] = df["isAlarm"].astype(int)
    else:
        pass
           
        

    ##################################################################
    # End
    ##################################################################
    
    return df

In [18]:
# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile, prefix, True)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

0
0
Execution time in seconds: 0.11801958084106445
Execution time in minutes: 0.0019669930140177407
Execution time in hours: 3.278321690029568e-05


In [19]:
testDF.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,ServiceID,CarID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
0,+,-1312080206,54879,0,:DBG:COMS:CCTS_0001:CAMS_0602,1,0,1,3,1,0,2021-12-01 08:59:57.697501952,2021-12-01 08:59:57.697501952,2021-12-01 08:59:57.697501952,40,8,OCCCMS,1,COM/DBG/1/CAM44,DBG:602 B2ES07,Status,FAILURE,<NA>,ES,Status,<NA>,<NA>,<NA>,COM,CAM,2021-12-01 08:59:58.251,2021-12-01 08:59:58.251,2021-12-01 08:59:58.251753216,0


In [20]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ENTRY_CODE_SUFFIX         1 non-null      string        
 1   ENTRY_CODE                1 non-null      string        
 2   ALARM_ID                  1 non-null      object        
 3   USER_ID                   1 non-null      object        
 4   EQUIPMENT_NAME            1 non-null      string        
 5   VALUE                     1 non-null      object        
 6   VALUE_STATE               1 non-null      category      
 7   ACKNOWLEDGEMENT_REQUIRED  1 non-null      int32         
 8   SEVERITY                  1 non-null      category      
 9   HIDDEN                    1 non-null      int32         
 10  THEME                     1 non-null      category      
 11  EQUIPMENT_DATE            1 non-null      datetime64[ns]
 12  ACQUISITION_DATE          

In [21]:
testDF["VALUE"]

0    1
Name: VALUE, dtype: object

In [22]:
testFile2 = fileList[1]
testFile2

# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile2, prefix)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

0
0
Execution time in seconds: 0.10003185272216797
Execution time in minutes: 0.0016671975453694662
Execution time in hours: 2.778662575615777e-05


In [23]:
testFile2

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\EventLogs\\EventListTest_1638320400_251602'

In [24]:
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\EventLogs\\EventListTest_1638320398_251753'

In [25]:
testDF.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,ASSET_DESC_CAT,EVENT_DESC_CAT,TrainID,ServiceID,CarID,AssetClass,AssetSubClass,DATETIME_SENT,DATETIME_RECEIVED,TIME_CODE,isAlarm
0,+,-1312080207,54880,0,:SER:COMS:CCTS_0001:CAMS_0208,1,0,True,3,True,0,2021-12-01 08:59:59.965480192,2021-12-01 08:59:59.965480192,2021-12-01 08:59:59.965480192,40,14,OCCCMS,1,COM/SER/B1/CAM20,SER:208 LWY1,Status,FAILURE,<NA>,LWY,Status,<NA>,<NA>,<NA>,COM,CAM,2021-12-01 09:00:00.251,2021-12-01 09:00:00.251,2021-12-01 09:00:00.251601920,False


In [26]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ENTRY_CODE_SUFFIX         1 non-null      string        
 1   ENTRY_CODE                1 non-null      string        
 2   ALARM_ID                  1 non-null      object        
 3   USER_ID                   1 non-null      object        
 4   EQUIPMENT_NAME            1 non-null      string        
 5   VALUE                     1 non-null      object        
 6   VALUE_STATE               1 non-null      category      
 7   ACKNOWLEDGEMENT_REQUIRED  1 non-null      bool          
 8   SEVERITY                  1 non-null      category      
 9   HIDDEN                    1 non-null      bool          
 10  THEME                     1 non-null      category      
 11  EQUIPMENT_DATE            1 non-null      datetime64[ns]
 12  ACQUISITION_DATE          

## Set Up SQL Server Database & Functions
The goal is to dump the pre-processed file into a SQL Server Database where further operations may then be made to perform the alarm tagging and nuisance event tagging. But first we will need to set up the SQL Server Database.

** Key tables **
1. EventList Table
2. AlarmList Table
3. Output_AlarmTagged Table
4. Output_TagComplete Table

In [27]:
# Open a database connection to target database
# All subsequent functions will depend on this connection
# Remember to close connection when done
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=SBSR-RD-0K00200;'
                      'Database=IAMS_DBtest;'
                      'Trusted_Connection=yes;')

# Define Server Parameters to Initiate Connection Engine via SQL Alchemy
# This has the same values as the connection request
# Only if one uses the windows authentification method
# Otherwise, one will need to define "UID" (user ID) + "PWD" (password)
serverParams = urllib.parse.quote_plus('Driver={SQL Server};'
                                       'Server=SBSR-RD-0K00200;'
                                       'Database=IAMS_DBtest;'
                                       'Trusted_Connection=yes;'
                                       #"UID=user;"
                                       #"PWD=password"
                                      )

# Create cursor to work in database
# SQL auto commits transactions
cursor = conn.cursor()

In [28]:
# Initialise tables (empty)
if initDB == True:
    # Initialise eventList Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='eventList')
                    CREATE TABLE eventList (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise alarmList Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='alarmList')
                    CREATE TABLE alarmList (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED varchar(255) null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise Output_AlarmTagged Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Output_AlarmTagged')
                    CREATE TABLE Output_AlarmTagged (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise Output_TagComplete
    cursor.execute('''
                if not exists (select * from sys.tables where name='Output_TagComplete')
                    CREATE TABLE Output_TagComplete (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE()),
                        RepeatAlarm int null,
                        AltAlarm2 int null,
                        AltAlarm3 int null,
                        NuisanceAlarm int null
                    )
                    ''')
    
    # Initialise TestValues_Master Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='TestValues_Master')
                    CREATE TABLE TestValues_Master (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise TestValues_Alarm Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='TestValues_Alarm')
                    CREATE TABLE TestValues_Alarm (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise Test_AlarmTagged Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Test_AlarmTagged')
                    CREATE TABLE Test_AlarmTagged (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise Test_Final Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Test_Final')
                    CREATE TABLE Test_Final (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE()),
                        RepeatAlarm int null, 
                        ToggleEventA int null, 
                        ToggleEventB int null, 
                        NuisanceAlarm int null
                    )
                    ''')
    
    
    conn.commit()
else:
    pass

In [29]:
# Function to Inspect All Tables
def list_dbTables():
    # Check list of tables in db
    cursor.execute("SELECT table_name FROM INFORMATION_SCHEMA.TABLES ")
    print(cursor.fetchall())

# List tables in database
# Note: %memit" prefix is used to log peak memory uage
list_dbTables()

[('eventList', ), ('alarmList', ), ('Output_AlarmTagged', ), ('Output_TagComplete', ), ('AppendTest', ), ('CompareTest', )]


In [30]:
# Function to list contents in target table
def list_tableContents(targetTable, rowLimit = 10):
    # Gets first n rows (rowLimit) from target table sorted by datetime (oldest entry first)
    # Not allowed to get all values as the table size can be huge
    cursor.execute(f"SELECT top {rowLimit} * FROM {targetTable} ORDER BY 'DATEANDTIME' ASC")
    results = cursor.fetchall()
    print(targetTable, "Contents")
    counter = 0
    print("Table Values")
    print("---START---")
    for row in results:
        counter=counter+1
        print(counter, row)
    print("---END---")

# Define the target table
# This table will be where the cleaned data would be saved to 
# and manipulated for future operations
targetTable = "AppendTest"

# Inspect target table
list_tableContents(targetTable)

AppendTest Contents
Table Values
---START---
---END---


In [31]:
# Function to list contents in target table
def list_tableColDtype(targetTable):
    cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM information_schema.columns where TABLE_NAME = '{targetTable}'")
    results = cursor.fetchall()
    print(targetTable, "Contents")
    counter = 0
    print("Table Column Data Type")
    print("---START---")
    for row in results:
        counter=counter+1
        print(counter, row)
    print("---END---")

# Inspect target table
targetTable = "AppendTest"
list_tableColDtype(targetTable)

AppendTest Contents
Table Column Data Type
---START---
1 ('eventID', 'int')
2 ('DATEANDTIME', 'datetime')
3 ('eventDesc', 'text')
4 ('TnF', 'int')
5 ('blanCol', 'int')
6 ('DATETIME_LOADED', 'datetime')
---END---


In [32]:
# Function to Create a dataframe with dummy data for testing purposes
def createDummyDataDF(inputData = "A"):
    if (inputData == "A"):
        dummyData = {
                     'eventID': [1, 2, 3],
                     'DATEANDTIME': [1644223829, 1644310229, 1644396629],
                     'eventDesc': ['Alpha', 'Bravo', 'Charlie'],
                     'TnF': [1, 1, 1] 
                    }
    else:
        dummyData = {
                     'eventID': [4, 5, 6],
                     'DATEANDTIME': [1644223829, 1644310229, 1644396629],
                     'eventDesc': ['Delta', 'Echo', 'Foxtrot'],
                     'TnF': [0, 0, 0] 
                    }
    
    df = pd.DataFrame(dummyData)
    df['DATEANDTIME'] = pd.to_datetime(df['DATEANDTIME'], unit='s')

    # To get time in seconds resolution if it comes in higher resolutions
    # Not required
    #df['TIME_S'] = df['DATEANDTIME'].dt.floor("s").dt.time

    return df

# Generate test dataframe in testing mode
if (testMode == True):
    # Create a dummy dataframe for testing purposes
    testDF = createDummyDataDF()
    print("Dataframe columns would have the same data type as SQL table values")
    testDF.info()
else:
    print("Test script skipped")

Dataframe columns would have the same data type as SQL table values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   eventID      3 non-null      int64         
 1   DATEANDTIME  3 non-null      datetime64[ns]
 2   eventDesc    3 non-null      object        
 3   TnF          3 non-null      int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 224.0+ bytes


In [33]:
# Inspect test dataframe
if (testMode == True):
    # Inspect Data
    print(testDF.head())
else:
    print("Test script skipped")

   eventID         DATEANDTIME eventDesc  TnF
0        1 2022-02-07 08:50:29     Alpha    1
1        2 2022-02-08 08:50:29     Bravo    1
2        3 2022-02-09 08:50:29   Charlie    1


In [34]:
# Function to append data to database as an entire dataframe
def appendData(tableName, inputDF, serverParams):    
    # Create connection engine 
    # Default connection function only works for SQLite
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(serverParams))
    # Append data
    inputDF.to_sql(tableName, con=engine, if_exists="append", index = False)

In [35]:
# Test data append function
if (testMode == True):
    # Test if append dataframe function works
    appendData("AppendTest", testDF, serverParams)
    list_tableContents("AppendTest")
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
1 (1, datetime.datetime(2022, 2, 7, 8, 50, 29), 'Alpha', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 54, 827000))
2 (2, datetime.datetime(2022, 2, 8, 8, 50, 29), 'Bravo', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 54, 830000))
3 (3, datetime.datetime(2022, 2, 9, 8, 50, 29), 'Charlie', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 54, 830000))
---END---


In [36]:
# Function to delete last n rows in table sorted by datetime (oldest first)
def delDataNRow(targetTable, nRow=3):
    cursor.execute(f"WITH CTE AS (SELECT TOP {nRow} * FROM {targetTable} ORDER BY DATEANDTIME DESC) DELETE FROM CTE")
    conn.commit()
    
# Function to delete all rows in table
def delDataAll(targetTable):
    cursor.execute(f"DELETE FROM {targetTable}")
    conn.commit()

In [37]:
# Test data delete latest nRows function
if (testMode == True):
    delDataNRow(targetTable, nRow=2)
    list_tableContents(targetTable)
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
1 (1, datetime.datetime(2022, 2, 7, 8, 50, 29), 'Alpha', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 54, 827000))
---END---


In [38]:
# Test data delete all function
if (resetTableData == True):
    delDataAll(targetTable)
    list_tableContents(targetTable)
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
---END---


In [39]:
# Delete redundant variables used in test
if (resetTableData == True):
    del testDF
else:
    print("Test script skipped")

## Push Raw Logs to SQL Server

In [40]:
# Define function to transfer files which has been processed successfully to an archive folder
def fileTransfer(file_name, dst_folder, src_folder=""):
    # If current folder is the source directory for the files, leave src_folder empty
    # check if file exist in destination
    if os.path.exists(dst_folder + file_name):
        # Split name and file type extension
        data = os.path.splitext(file_name)
        only_name = data[0]
        extension = data[1]
        # Adding the new name
        new_base = only_name + '_new' + extension
        # construct full file path
        new_name = os.path.join(dst_folder, new_base)
        # move file
        shutil.move(src_folder + file_name, new_name)
    else:
        shutil.move(src_folder + file_name, dst_folder + file_name)
    print(file_name, "Transferred to destination folder", dst_folder)

In [41]:
# Get timestamp of operation start:
dateTimeStartOp = dt.datetime.now()
print("Start Operation")
print(dateTimeStartOp)

# Get list of files to initialise
fileList = os.listdir()
# Ignore temp files (large files not fully downloaded)
fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]

# Define Archive Folder Location and Target table
if (prefix == 'TestAE'):
    targetTable = "eventList"
    archiveFolderLoc = "../EventLogs_archive/"
else:
    targetTable = "alarmList"
    archiveFolderLoc = "../AlarmLogs_archive/"
    
# Define wait time for new files to be added
# If there are no more files within directory
# Else the script will be terminated
# Time is in seconds
waitTimeInterval = 1
waitTimeLimit = 10

while (len(fileList) > 0):
    # Initiate batch run  
    print("Start batch run")
    # Get timestamp of batch run start:
    dateTimeStartBatch = dt.datetime.now()
    print(dateTimeStartBatch)
    
    for file in fileList:
        # Get Oldest File in Directory in a First in First Out (FIFO) manner as an input
        inputFile = min(fileList, key=os.path.getctime)
        
        #############################################
        # Process file as a cleaned dataframe - START
        #############################################
        
        # Get timestamp of run start:
        print("Commence processing of", inputFile)
        dateTimeStartRun = dt.datetime.now()
        print(dateTimeStartRun)
        
        # Ingest log files as a dataframe for further processing
        df = cleaningScript_vector(inputFile, prefix, True)
        
        #############################################
        # Process file as a cleaned dataframe - END
        #############################################
        
        # Append data to SQL DB
        appendData(targetTable, df, serverParams)

        # Archive processed file
        fileTransfer(inputFile, archiveFolderLoc)
        #print(inputFile, "archived")
        
        # Update list of files
        fileList = os.listdir()
        # Ignore temp files (large files not fully downloaded)
        fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]
        
        # Get timestamp of run end: 
        dateTimeEndRun = dt.datetime.now()
        print("Run Tlapsed Time")
        print(dateTimeEndRun - dateTimeStartRun)
        del dateTimeEndRun, dateTimeStartRun
        
    print("Batch run completed")
    # Get timestamp of batch run end:
    dateTimeEndBatch = dt.datetime.now()
    print(dateTimeEndBatch)
    print("Batch Elapsed Time")
    print(dateTimeEndBatch - dateTimeStartBatch)
    del dateTimeEndBatch, dateTimeStartBatch
    
    # Check if new files are added to restart process
    timeCounter = 0
    while (len(fileList) == 0):
        # When initial batch run is completed check again for newly added files
        # This is to account for any lag or latency in the file transfer
        # n second(s) timer delay between batches
        time.sleep(waitTimeInterval)
        timeCounter = timeCounter + waitTimeInterval
        
        # Update list of files
        fileList = os.listdir()
        # Ignore temp files (large files not fully downloaded)
        fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]

        # Terminate script if wait too long
        if (timeCounter > waitTimeLimit):
            break
        
    
print("Script Terminated")
dateTimeEndOp = dt.datetime.now()
print(dateTimeEndOp)
print("Operation Elapsed Time")
print(dateTimeEndOp - dateTimeStartOp)

Start Operation
2022-04-07 11:11:54.926566
Start batch run
2022-04-07 11:11:54.933568
Commence processing of EventListTest_1638320398_251753
2022-04-07 11:11:55.069566
0
0
EventListTest_1638320398_251753 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.130000
Commence processing of EventListTest_1638320400_251602
2022-04-07 11:11:55.338567
0
0
EventListTest_1638320400_251602 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.128000
Commence processing of EventListTest_1638320402_253507
2022-04-07 11:11:55.606566
0
0
EventListTest_1638320402_253507 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:04.217186
Commence processing of EventListTest_1638320403_252577
2022-04-07 11:11:59.957780
0
0
EventListTest_1638320403_252577 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.125000
Commence processing of EventListTest_1638320404_252624
2022-04-07 11:12:00.219780
0
0
Even

EventListTest_1638320445_252722 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.123002
Commence processing of EventListTest_1638320446_252881
2022-04-07 11:12:22.719560
0
0
EventListTest_1638320446_252881 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.123001
Commence processing of EventListTest_1638320447_252663
2022-04-07 11:12:22.977560
0
0
EventListTest_1638320447_252663 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.132001
Commence processing of EventListTest_1638320448_252652
2022-04-07 11:12:23.241560
0
0
EventListTest_1638320448_252652 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:04.214054
Commence processing of EventListTest_1638320449_252618
2022-04-07 11:12:27.592611
0
0
EventListTest_1638320449_252618 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.132000
Commence processing of EventListTest_1638320450_252605
2

EventListTest_1638320491_254252 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.141972
Commence processing of EventListTest_1638320492_252785
2022-04-07 11:12:45.691377
0
0
EventListTest_1638320492_252785 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.159982
Commence processing of EventListTest_1638320493_252720
2022-04-07 11:12:45.979393
0
0
EventListTest_1638320493_252720 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.132964
Commence processing of EventListTest_1638320495_251722
2022-04-07 11:12:46.248407
0
0
EventListTest_1638320495_251722 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.122950
Commence processing of EventListTest_1638320496_252639
2022-04-07 11:12:46.501358
0
0
EventListTest_1638320496_252639 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.124999
Commence processing of EventListTest_1638320497_251685
2

EventListTest_1638320534_251981 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.141002
Commence processing of EventListTest_1638320535_251945
2022-04-07 11:12:56.659529
0
0
EventListTest_1638320535_251945 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.137001
Commence processing of EventListTest_1638320536_252181
2022-04-07 11:12:56.928530
0
0
EventListTest_1638320536_252181 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.133004
Commence processing of EventListTest_1638320537_253850
2022-04-07 11:12:57.195563
0
0
EventListTest_1638320537_253850 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.140967
Commence processing of EventListTest_1638320539_251840
2022-04-07 11:12:57.475529
0
0
EventListTest_1638320539_251840 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.132000
Commence processing of EventListTest_1638320540_251854
2

EventListTest_1638320577_254063 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.129030
Commence processing of EventListTest_1638320578_254116
2022-04-07 11:13:07.783531
0
0
EventListTest_1638320578_254116 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.132000
Commence processing of EventListTest_1638320579_254101
2022-04-07 11:13:08.052575
0
0
EventListTest_1638320579_254101 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.142954
Commence processing of EventListTest_1638320580_254098
2022-04-07 11:13:08.338532
0
0
EventListTest_1638320580_254098 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.145998
Commence processing of EventListTest_1638320581_254362
2022-04-07 11:13:08.619565
0
0
EventListTest_1638320581_254362 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.127003
Commence processing of EventListTest_1638320582_254011
2

...

EventListTest_1638323873_254003 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.116001
Commence processing of EventListTest_1638323874_251997
2022-04-07 11:23:54.411451
0
0
EventListTest_1638323874_251997 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.118003
Commence processing of EventListTest_1638323875_252180
2022-04-07 11:23:54.537453
0
0
EventListTest_1638323875_252180 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.127001
Commence processing of EventListTest_1638323876_253950
2022-04-07 11:23:54.671452
0
0
EventListTest_1638323876_253950 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.143000
Commence processing of EventListTest_1638323877_251955
2022-04-07 11:23:54.821452
0
0
EventListTest_1638323877_251955 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.128000
Commence processing of EventListTest_1638323878_253036
2

EventListTest_1638323921_252019 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.131000
Commence processing of EventListTest_1638323922_251997
2022-04-07 11:23:59.604988
0
0
EventListTest_1638323922_251997 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.117000
Commence processing of EventListTest_1638323923_251983
2022-04-07 11:23:59.728987
0
0
EventListTest_1638323923_251983 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.115001
Commence processing of EventListTest_1638323924_251954
2022-04-07 11:23:59.849991
0
0
EventListTest_1638323924_251954 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.119996
Commence processing of EventListTest_1638323925_251941
2022-04-07 11:23:59.975987
0
0
EventListTest_1638323925_251941 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.121001
Commence processing of EventListTest_1638323927_251880
2

EventListTest_1638323966_252109 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.122985
Commence processing of EventListTest_1638323967_252161
2022-04-07 11:24:04.670850
0
0
EventListTest_1638323967_252161 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.118001
Commence processing of EventListTest_1638323968_252134
2022-04-07 11:24:04.792850
0
0
EventListTest_1638323968_252134 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.115000
Commence processing of EventListTest_1638323969_252025
2022-04-07 11:24:04.910850
0
0
EventListTest_1638323969_252025 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.130001
Commence processing of EventListTest_1638323970_254011
2022-04-07 11:24:05.043850
0
0
EventListTest_1638323970_254011 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.129000
Commence processing of EventListTest_1638323971_255124
2

EventListTest_1638324008_252410 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.142995
Commence processing of EventListTest_1638324009_252374
2022-04-07 11:24:10.085361
0
0
EventListTest_1638324009_252374 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.151999
Commence processing of EventListTest_1638324010_252590
2022-04-07 11:24:10.239360
0
0
EventListTest_1638324010_252590 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.159000
Commence processing of EventListTest_1638324011_253692
2022-04-07 11:24:10.400360
0
0
EventListTest_1638324011_253692 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.177001
Commence processing of EventListTest_1638324012_254217
2022-04-07 11:24:10.580360
0
0
EventListTest_1638324012_254217 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.141001
Commence processing of EventListTest_1638324014_254368
2

EventListTest_1638324051_253757 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.125000
Commence processing of EventListTest_1638324052_252495
2022-04-07 11:24:15.730102
0
0
EventListTest_1638324052_252495 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.128999
Commence processing of EventListTest_1638324053_252496
2022-04-07 11:24:15.859101
0
0
EventListTest_1638324053_252496 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.126001
Commence processing of EventListTest_1638324054_252389
2022-04-07 11:24:15.986111
0
0
EventListTest_1638324054_252389 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.148996
Commence processing of EventListTest_1638324055_252697
2022-04-07 11:24:16.135107
0
0
EventListTest_1638324055_252697 Transferred to destination folder ../EventLogs_archive/
Run Tlapsed Time
0:00:00.145994
Commence processing of EventListTest_1638324056_252420
2

In [42]:
# Close Connection to Database
conn.close()